In [1]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np

from SVGFuncAnimation import SVGFuncAnimation

plt.rcParams['animation.frame_format'] = 'svg'
plt.rcParams['animation.html'] = 'jshtml'
plt.rcParams['animation.embed_limit'] = 2 ** 128

In [2]:
%%time

# Test the standard FuncAnimation + HTMLWriter (implicitly used by to_jshtml)

np.random.seed(0)

def update_line(num, data, line):
    line.set_data(range(num), data[:num])
    return line,


size = 1000
fig = plt.figure()
data = np.random.rand(size)
l, = plt.plot([], [], 'r-')
plt.xlim(0, size - 1)
plt.ylim(0, 1)

anim = FuncAnimation(fig, update_line, range(1, size + 1), fargs=(data, l),
                           interval=150, blit=True)
plt.close(fig)

print(f'Size: {len(anim.to_jshtml()):,}')

Size: 31,604,895
Wall time: 32.5 s


In [3]:
%%time

# Test SVGFuncAnimation (which uses it's own, built-in Writer)

np.random.seed(0)

def update_line(num, data, line):
    line.set_data(range(num), data[:num])
    return line,

size = 1000
fig = plt.figure()
data = np.random.rand(size)
l, = plt.plot([], [], 'r-')
plt.xlim(0, size - 1)
plt.ylim(0, 1)

svganim = SVGFuncAnimation(fig, update_line, range(1, size + 1), fargs=(data, l))

plt.close(fig)

print(f'Size: {len(svganim.to_jshtml()):,}')

Size: 12,099,145
Wall time: 1.04 s
